## Wrapping a function in a thread

In [5]:
import threading
import time

def my_func(index):
    
    print("inside my_func at index {0}".format(index))
    time.sleep(2)
    print("exiting my_func at index {0}".format(index))
    
    

    
    

In [13]:
print("Main : before creating thread")
x = threading.Thread(target = my_func, args = (1,))
print("Main : before starting thread")
x.start()
print("Main : waiting for thread to finish")
x.join()
print("Main : all done")

Main : before creating thread
Main : before starting thread
inside my_func at index 1
Main : waiting for thread to finish
exiting my_func at index 1
Main : all done


In [12]:
print("Main : before creating thread")
x = threading.Thread(target = my_func, args = (1,), daemon=True)
print("Main : before starting thread")
x.start()
print("Main : waiting for thread to finish")
print("Main : all done")

Main : before creating thread
Main : before starting thread
inside my_func at index 1
Main : waiting for thread to finish
Main : all done
exiting my_func at index 1


## Running multiple threads

### Method 1

In [18]:
threads = list()
for index in range(3):
    print("Main    : create and start thread {}".format(index))
    x = threading.Thread(target=my_func, args=(index,))
    threads.append(x)
    x.start()

for index, thread in enumerate(threads):
    print("Main    : before joining thread {}".format(index))
    ## thread.join()
    print("Main    : thread {} done".format(index))

Main    : create and start thread 0
inside my_func at index 0
Main    : create and start thread 1
inside my_func at index 1
Main    : create and start thread 2
inside my_func at index 2
Main    : before joining thread 0
Main    : thread 0 done
Main    : before joining thread 1
Main    : thread 1 done
Main    : before joining thread 2
Main    : thread 2 done
exiting my_func at index 2
exiting my_func at index 0
exiting my_func at index 1


In [16]:
print("hi")

hi


### Method 2

In [21]:
import concurrent.futures

In [23]:
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    executor.map(my_func, range(3))

inside my_func at index 0
inside my_func at index 1
exiting my_func at index 1exiting my_func at index 0
inside my_func at index 2

exiting my_func at index 2


## Race condition

In [24]:
## Create a fake database which is accessed by threads

In [39]:
import threading
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()


    def update(self, name):
        print("Thread {}: starting update".format(name))
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        print("Thread {}: finishing update".format(name))
        
    def locked_update(self, name):
        print("Locked Thread {}: starting update".format(name))
        print("Thread {} about to lock: starting update".format(name))
        with self._lock:
            print("Thread {} has lock".format(name))
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.debug("Thread {} about to release lock".format(name))
        print("Thread {} after release".format(name))
        print("Thread {} finishing update".format(name))

In [40]:
a = FakeDatabase()

In [37]:
a.update("hi")

Thread hi: starting update
Thread hi: finishing update


In [31]:
logging.info()

TypeError: info() missing 1 required positional argument: 'msg'

In [38]:
database = FakeDatabase()
print("Testing update. Starting value is {}".format(database.value))
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
print("Testing update. Ending value is {}".format(database.value))

Testing update. Starting value is 0
Thread 0: starting update
Thread 1: starting update
Thread 0: finishing update
Thread 1: finishing update
Testing update. Ending value is 1


1 when it should be 2 as threads concurrently accessed the DB and incremented independently , race condition !!

## How do you solve this ?

In [ ]:
Using locks - mutex## add a lock update in fake database above

In [41]:
## without lock
database = FakeDatabase()
print("Testing update. Starting value is {}".format(database.value))
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
print("Testing update. Ending value is {}".format(database.value))

Testing update. Starting value is 0
Thread 0: starting update
Thread 1: starting update
Thread 1: finishing update
Thread 0: finishing update
Testing update. Ending value is 1


In [42]:
## With lock
database = FakeDatabase()
print("Testing update. Starting value is {}".format(database.value))
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.locked_update, index)
print("Testing update. Ending value is {}".format(database.value))

Testing update. Starting value is 0
Locked Thread 0: starting update
Thread 0 about to lock: starting update
Thread 0 has lock
Locked Thread 1: starting update
Thread 1 about to lock: starting update
Thread 0 after release
Thread 0 finishing update
Thread 1 has lock
Thread 1 after release
Thread 1 finishing update
Testing update. Ending value is 2
